# Cleaning raw scraped data

Open raw scraped data from `.csv` and clean it.

Imports.

In [83]:
from itertools import combinations

import pandas as pd

Pathing to the data.

In [84]:
data_path = '../data'
raw_data_path = f'{data_path}/csv/raw'
clean_data_path = f'{data_path}/csv/clean'

Open scraped data.

In [85]:
matches_df = pd.read_csv(f'{raw_data_path}/bundesliga_matches.csv')

See the shape of the scraped data - rows and columns.

In [86]:
matches_df.shape

(4425, 239)

## Standardise column names

See all column names of the scraped data.

In [87]:
matches_df.columns.tolist()

['Date',
 'Time',
 'Comp',
 'Round',
 'Day',
 'Venue',
 'Result',
 'GF',
 'GA',
 'Opponent',
 'xG',
 'xGA',
 'Poss',
 'Attendance',
 'Captain',
 'Formation',
 'Referee',
 'Match Report',
 'Notes',
 'Team',
 'shooting  Comp',
 'shooting  Round',
 'shooting  Day',
 'shooting  Venue',
 'shooting  Result',
 'shooting  GF',
 'shooting  GA',
 'shooting  Opponent',
 'shooting Standard Gls',
 'shooting Standard Sh',
 'shooting Standard SoT',
 'shooting Standard SoT%',
 'shooting Standard G/Sh',
 'shooting Standard G/SoT',
 'shooting Standard Dist',
 'shooting Standard FK',
 'shooting Standard PK',
 'shooting Standard PKatt',
 'shooting Expected xG',
 'shooting Expected npxG',
 'shooting Expected npxG/Sh',
 'shooting Expected G-xG',
 'shooting Expected np:G-xG',
 'shooting Unnamed: 25_level_0 Match Report',
 'keeper  Comp',
 'keeper  Round',
 'keeper  Day',
 'keeper  Venue',
 'keeper  Result',
 'keeper  GF',
 'keeper  GA',
 'keeper  Opponent',
 'keeper Performance SoTA',
 'keeper Performance GA

There are numerous issues with the column names:
- Columns that have double spaces in their names. Replace them with single spaces.
- Columns do not follow a consistent naming convention. Convert all column names to lowercase.
- Columns that have spaces in their names. Replace them with underscores.
- There are numerous columns from each category where there is `_unnamed_<some number>_level_0`. Remove the `_unnamed_<some number>_level_0` from the column names.

In [88]:
matches_df.columns = matches_df.columns.str.replace('  ', ' ')
matches_df.columns = matches_df.columns.str.lower()
matches_df.columns = matches_df.columns.str.replace(' ', '_')
matches_df.columns = matches_df.columns.str.replace(r'(.*)_unnamed:_\d+_level_0_(.*)', r'\1_\2', regex=True)
matches_df.columns.tolist()

['date',
 'time',
 'comp',
 'round',
 'day',
 'venue',
 'result',
 'gf',
 'ga',
 'opponent',
 'xg',
 'xga',
 'poss',
 'attendance',
 'captain',
 'formation',
 'referee',
 'match_report',
 'notes',
 'team',
 'shooting_comp',
 'shooting_round',
 'shooting_day',
 'shooting_venue',
 'shooting_result',
 'shooting_gf',
 'shooting_ga',
 'shooting_opponent',
 'shooting_standard_gls',
 'shooting_standard_sh',
 'shooting_standard_sot',
 'shooting_standard_sot%',
 'shooting_standard_g/sh',
 'shooting_standard_g/sot',
 'shooting_standard_dist',
 'shooting_standard_fk',
 'shooting_standard_pk',
 'shooting_standard_pkatt',
 'shooting_expected_xg',
 'shooting_expected_npxg',
 'shooting_expected_npxg/sh',
 'shooting_expected_g-xg',
 'shooting_expected_np:g-xg',
 'shooting_match_report',
 'keeper_comp',
 'keeper_round',
 'keeper_day',
 'keeper_venue',
 'keeper_result',
 'keeper_gf',
 'keeper_ga',
 'keeper_opponent',
 'keeper_performance_sota',
 'keeper_performance_ga',
 'keeper_performance_saves',
 'ke

## Drop columns

Display data types.

In [89]:
matches_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4425 entries, 0 to 4424
Data columns (total 239 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    date                            4425 non-null   object 
 1    time                            4425 non-null   object 
 2    comp                            4425 non-null   object 
 3    round                           4425 non-null   object 
 4    day                             4425 non-null   object 
 5    venue                           4425 non-null   object 
 6    result                          4425 non-null   object 
 7    gf                              4425 non-null   object 
 8    ga                              4425 non-null   object 
 9    opponent                        4425 non-null   object 
 10   xg                              4039 non-null   float64
 11   xga                             4039 non-null   float64
 12   poss              

At first sight, the data types seem favorable. However, there is a big amount of `object`(`string`) data.

Explore first and last five entries.

In [90]:
matches_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,misc_performance_int,misc_performance_tklw,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%,misc_match_report
0,2020-09-12,15:30,DFB-Pokal,First round,Sat,Away,W,2,1,1860 Munich,...,7.0,8.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,Match Report
1,2020-09-19,15:30,Bundesliga,Matchweek 1,Sat,Home,D,1,1,Arminia,...,11.0,12.0,0.0,0.0,0.0,63.0,17.0,15.0,53.1,Match Report
2,2020-09-25,20:30,Bundesliga,Matchweek 2,Fri,Away,W,3,1,Hertha BSC,...,8.0,12.0,1.0,0.0,1.0,57.0,23.0,12.0,65.7,Match Report
3,2020-10-03,15:30,Bundesliga,Matchweek 3,Sat,Home,W,2,1,Hoffenheim,...,7.0,6.0,0.0,0.0,0.0,56.0,24.0,9.0,72.7,Match Report
4,2020-10-18,15:30,Bundesliga,Matchweek 4,Sun,Away,D,1,1,Köln,...,7.0,10.0,1.0,0.0,0.0,65.0,36.0,32.0,52.9,Match Report


In [91]:
matches_df.tail()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,misc_performance_int,misc_performance_tklw,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%,misc_match_report
4420,2022-04-16,18:30,Bundesliga,Matchweek 30,Sat,Away,W,3,1,M'Gladbach,...,12.0,12.0,0.0,0.0,0.0,63.0,16.0,14.0,53.3,Match Report
4421,2022-04-23,15:30,Bundesliga,Matchweek 31,Sat,Home,W,3,1,Arminia,...,6.0,12.0,0.0,0.0,1.0,63.0,20.0,18.0,52.6,Match Report
4422,2022-04-30,15:30,Bundesliga,Matchweek 32,Sat,Away,W,4,1,Augsburg,...,9.0,16.0,1.0,0.0,0.0,54.0,15.0,21.0,41.7,Match Report
4423,2022-05-07,15:30,Bundesliga,Matchweek 33,Sat,Home,L,0,1,Wolfsburg,...,11.0,9.0,0.0,0.0,0.0,57.0,18.0,25.0,41.9,Match Report
4424,2022-05-14,15:30,Bundesliga,Matchweek 34,Sat,Away,L,1,2,Stuttgart,...,2.0,14.0,0.0,1.0,0.0,52.0,7.0,24.0,22.6,Match Report


Drop the `notes` and `match_report` columns as they do not provide any useful information.

In [92]:
matches_df.drop(['notes', 'match_report'], axis=1, inplace=True)

Find duplicate columns.

The following code was borrowed from [https://stackoverflow.com/a/58002867/9553927](https://stackoverflow.com/a/58002867/9553927).

In [93]:
def get_list_of_equal_cols(dataframe):
    return[(i, j) for i,j in combinations(dataframe, 2) if dataframe[i].equals(dataframe[j])]

In [94]:
get_list_of_equal_cols(matches_df)

[('comp', 'shooting_comp'),
 ('comp', 'keeper_comp'),
 ('comp', 'passing_comp'),
 ('comp', 'passing_types_comp'),
 ('comp', 'gca_comp'),
 ('comp', 'defense_comp'),
 ('comp', 'possession_comp'),
 ('comp', 'misc_comp'),
 ('round', 'shooting_round'),
 ('round', 'keeper_round'),
 ('round', 'passing_round'),
 ('round', 'passing_types_round'),
 ('round', 'gca_round'),
 ('round', 'defense_round'),
 ('round', 'possession_round'),
 ('round', 'misc_round'),
 ('day', 'shooting_day'),
 ('day', 'keeper_day'),
 ('day', 'passing_day'),
 ('day', 'passing_types_day'),
 ('day', 'gca_day'),
 ('day', 'defense_day'),
 ('day', 'possession_day'),
 ('day', 'misc_day'),
 ('venue', 'shooting_venue'),
 ('venue', 'keeper_venue'),
 ('venue', 'passing_venue'),
 ('venue', 'passing_types_venue'),
 ('venue', 'gca_venue'),
 ('venue', 'defense_venue'),
 ('venue', 'possession_venue'),
 ('venue', 'misc_venue'),
 ('result', 'shooting_result'),
 ('result', 'keeper_result'),
 ('result', 'passing_result'),
 ('result', 'passin

`comp`, `round`, `day`, `venue`, `result`, `opponent`, `match_report`, `gf` and `ga` have a lot of duplicates.

Drop all duplicate columns.

In [95]:
regex_of_columns_to_remove = '.+_(comp|round|day|venue|result|opponent|match_report|gf|ga)'
matches_df.drop(matches_df.filter(regex=regex_of_columns_to_remove).columns, axis=1, inplace=True)

In [96]:
remaining_equal_cols = get_list_of_equal_cols(matches_df)
remaining_equal_cols

[('xg', 'shooting_expected_xg'),
 ('poss', 'possession_poss'),
 ('passing_total_cmp', 'passing_types_outcomes_cmp'),
 ('passing_total_att', 'passing_types_att'),
 ('passing_types_pass_types_crs', 'misc_performance_crs'),
 ('defense_tackles_tklw', 'misc_performance_tklw'),
 ('defense_int', 'misc_performance_int')]

Pick the second item (column) from each tuple and drop it from the dataframe.

In [97]:
cols_to_drop = [item[1] for item in remaining_equal_cols]
matches_df.drop(cols_to_drop, axis=1, inplace=True)

See data types again.

In [98]:
matches_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4425 entries, 0 to 4424
Data columns (total 157 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    date                            4425 non-null   object 
 1    time                            4425 non-null   object 
 2    comp                            4425 non-null   object 
 3    round                           4425 non-null   object 
 4    day                             4425 non-null   object 
 5    venue                           4425 non-null   object 
 6    result                          4425 non-null   object 
 7    gf                              4425 non-null   object 
 8    ga                              4425 non-null   object 
 9    opponent                        4425 non-null   object 
 10   xg                              4039 non-null   float64
 11   xga                             4039 non-null   float64
 12   poss              

## Convert data types

Out of the 14 `object(string)` dtype columns, `gf` and `ga` are supposed to be numeric(`float` or `int`).

Check the unique values of `gf`.

In [99]:
matches_df['gf'].unique().tolist()

['2',
 '1',
 '3',
 '0',
 '5',
 '4',
 '6',
 '1 (6)',
 '3 (5)',
 '2 (4)',
 '0.0',
 '1.0',
 '3.0',
 '8.0',
 '2.0',
 '4.0',
 '7',
 '1 (3)',
 '8',
 '2 (3)',
 '3 (2)',
 '2 (5)',
 '1 (5)',
 '4 (5)',
 '1 (2)',
 '1 (4)',
 '11',
 '2 (8)',
 '2 (6)',
 '2 (0)',
 '3 (3)',
 '3 (4)',
 '9',
 '12']

Show matches where `gf` was `2 (5)`.

In [100]:
matches_df[matches_df['gf'] == '2 (5)']

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%
1213,2017-08-05,20:30,DFL-Supercup,DFL-Supercup,Sat,Away,D,2 (5),2 (4),Dortmund,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2317,2021-01-13,20:45,DFB-Pokal,Second round,Wed,Away,D,2 (5),2 (6),Holstein Kiel,...,14.0,17.0,3.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3231,2018-08-20,18:30,DFB-Pokal,First round,Mon,Away,D,2 (5),2 (3),Energie Cottbus,...,19.0,16.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4261,2021-10-27,18:30,DFB-Pokal,Round of 32,Wed,Home,D,2 (5),2 (4),Augsburg,...,13.0,15.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


Upon looking on the Internet, it seems that the number in the parentheses is goals scored in the penalty shootout.

The nature of penalty shootouts is that they are not part of the match itself. Therefore, they should not be included in the `gf` column. Moreover, since the number of goals is always a whole number, get the number before the dot. Finally, convert both `gf` and `ga` to `int`.

In [101]:
matches_df['gf'] = matches_df['gf'].str.split(' ').str[0]
matches_df['gf'] = matches_df['gf'].str.split('.').str[0]
matches_df['gf'] = matches_df['gf'].astype('int')

matches_df['ga'] = matches_df['ga'].str.split(' ').str[0]
matches_df['ga'] = matches_df['ga'].str.split('.').str[0]
matches_df['ga'] = matches_df['ga'].astype('int')

## Look at basic statistics

Show basic statistics.

In [102]:
matches_df.describe()

,gf,ga,xg,xga,poss,attendance,shooting_standard_gls,shooting_standard_sh,shooting_standard_sot,shooting_standard_sot%,...,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%
count,4425.000000,4425.000000,4039.000000,4039.000000,4393.000000,3560.000000,4413.000000,4393.000000,4393.000000,4391.000000,...,4392.000000,4392.000000,4392.000000,4043.000000,4043.000000,4410.000000,4039.000000,4039.000000,4039.000000,4039.000000
mean,1.630960,1.503729,1.489354,1.456078,50.916913,37084.585112,1.587809,13.222627,4.635101,35.027465,...,12.401867,11.675546,2.018670,0.114519,0.154093,0.039002,54.479327,16.865066,16.834613,50.023595
std,1.419029,1.330559,0.852480,0.836276,11.446664,20263.103919,1.400138,5.472324,2.697823,15.335462,...,3.944321,3.869443,1.650964,0.335134,0.384960,0.195951,10.226509,6.948823,6.925692,10.242490
min,0.000000,0.000000,0.000000,0.000000,17.000000,100.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,13.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.900000,0.800000,43.000000,23703.000000,1.000000,9.000000,3.000000,25.000000,...,10.000000,9.000000,1.000000,0.000000,0.000000,0.000000,48.000000,12.000000,12.000000,43.300000
50%,1.000000,1.000000,1.300000,1.300000,51.000000,33305.000000,1.000000,13.000000,4.000000,33.300000,...,12.000000,11.000000,2.000000,0.000000,0.000000,0.000000,55.000000,16.000000,16.000000,50.000000
75%,2.000000,2.000000,2.000000,1.900000,59.000000,50000.000000,2.000000,16.000000,6.000000,44.400000,...,15.000000,14.000000,3.000000,0.000000,0.000000,0.000000,61.000000,21.000000,21.000000,57.100000
max,12.000000,8.000000,5.600000,5.600000,84.000000,90071.000000,11.000000,44.000000,21.000000,100.000000,...,29.000000,28.000000,10.000000,3.000000,3.000000,2.000000,93.000000,49.000000,49.000000,100.000000


See statistics about `object(string)` columns.

In [103]:
matches_df.describe(include=object)

,date,time,comp,round,day,venue,result,opponent,captain,formation,referee,team
count,4425,4425,4425,4425,4425,4425,4425,4425,4413,4413,4413,4425
unique,928,32,8,51,7,3,3,199,186,22,171,27
top,2022-05-14,15:30,Bundesliga,Group stage,Sat,Away,W,Bayern Munich,Manuel Neuer,4-2-3-1,Felix Brych,Bayern Munich
freq,18,2277,3682,243,2304,2273,1796,225,191,1175,228,297


A big portion of the matches are played in the `Bundesliga` but there are `8` competitions in the dataset.
`team` and `opponent` have very different values, probably due to the fact that matches outside of `Bundesliga` are included.

`venue` shows 3 unique values. `Home` and `Away` are obvious but check the third value.

In [104]:
matches_df['venue'].value_counts()

Away       2273
Home       2137
Neutral      15
Name: venue, dtype: int64

## Explore matches from `Bundesliga` and other competitions

In [107]:
bundesliga_matches_df = matches_df[matches_df['comp'] == 'Bundesliga'].head()
other_matches_df = matches_df[matches_df['comp'] != 'Bundesliga'].head()

combined_df = pd.concat([bundesliga_matches_df, other_matches_df])
combined_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%
1,2020-09-19,15:30,Bundesliga,Matchweek 1,Sat,Home,D,1,1,Arminia,...,14.0,11.0,2.0,0.0,0.0,0.0,63.0,17.0,15.0,53.1
2,2020-09-25,20:30,Bundesliga,Matchweek 2,Fri,Away,W,3,1,Hertha BSC,...,16.0,13.0,0.0,1.0,0.0,1.0,57.0,23.0,12.0,65.7
3,2020-10-03,15:30,Bundesliga,Matchweek 3,Sat,Home,W,2,1,Hoffenheim,...,12.0,7.0,9.0,0.0,0.0,0.0,56.0,24.0,9.0,72.7
4,2020-10-18,15:30,Bundesliga,Matchweek 4,Sun,Away,D,1,1,Köln,...,18.0,18.0,0.0,1.0,0.0,0.0,65.0,36.0,32.0,52.9
5,2020-10-24,15:30,Bundesliga,Matchweek 5,Sat,Away,L,0,5,Bayern Munich,...,12.0,10.0,7.0,0.0,0.0,0.0,66.0,12.0,8.0,60.0
0,2020-09-12,15:30,DFB-Pokal,First round,Sat,Away,W,2,1,1860 Munich,...,10.0,13.0,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
16,2021-01-12,20:45,DFB-Pokal,Second round,Tue,Away,L,1,4,Leverkusen,...,15.0,7.0,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
36,2018-08-20,18:30,DFB-Pokal,First round,Mon,Away,W,2,1,Braunschweig,...,8.0,13.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
46,2018-10-30,18:30,DFB-Pokal,Second round,Tue,Away,W,2,0,Darmstadt 98,...,14.0,11.0,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
58,2019-02-06,20:45,DFB-Pokal,Round of 16,Wed,Home,L,2,3,Bayern Munich,...,16.0,11.0,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


Matches from other competitions seem to have lots of `NaN` values.

Assign a `NaNs` column to show how many `NaN` values there are in each row.

In [108]:
combined_df.insert(loc=0, column='NaNs', value=combined_df.isnull().sum(axis=1))
combined_df

,NaNs,date,time,comp,round,day,venue,result,gf,ga,...,misc_performance_fls,misc_performance_fld,misc_performance_off,misc_performance_pkwon,misc_performance_pkcon,misc_performance_og,misc_performance_recov,misc_aerial_duels_won,misc_aerial_duels_lost,misc_aerial_duels_won%
1,0,2020-09-19,15:30,Bundesliga,Matchweek 1,Sat,Home,D,1,1,...,14.0,11.0,2.0,0.0,0.0,0.0,63.0,17.0,15.0,53.1
2,0,2020-09-25,20:30,Bundesliga,Matchweek 2,Fri,Away,W,3,1,...,16.0,13.0,0.0,1.0,0.0,1.0,57.0,23.0,12.0,65.7
3,0,2020-10-03,15:30,Bundesliga,Matchweek 3,Sat,Home,W,2,1,...,12.0,7.0,9.0,0.0,0.0,0.0,56.0,24.0,9.0,72.7
4,0,2020-10-18,15:30,Bundesliga,Matchweek 4,Sun,Away,D,1,1,...,18.0,18.0,0.0,1.0,0.0,0.0,65.0,36.0,32.0,52.9
5,1,2020-10-24,15:30,Bundesliga,Matchweek 5,Sat,Away,L,0,5,...,12.0,10.0,7.0,0.0,0.0,0.0,66.0,12.0,8.0,60.0
0,114,2020-09-12,15:30,DFB-Pokal,First round,Sat,Away,W,2,1,...,10.0,13.0,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
16,114,2021-01-12,20:45,DFB-Pokal,Second round,Tue,Away,L,1,4,...,15.0,7.0,6.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
36,113,2018-08-20,18:30,DFB-Pokal,First round,Mon,Away,W,2,1,...,8.0,13.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
46,114,2018-10-30,18:30,DFB-Pokal,Second round,Tue,Away,W,2,0,...,14.0,11.0,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
58,113,2019-02-06,20:45,DFB-Pokal,Round of 16,Wed,Home,L,2,3,...,16.0,11.0,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN


Matches from other competitions miss a lot of data. It's better to drop them. Even more so when the model will be focused on the `Bundesliga` matches.

In [109]:
matches_df = matches_df[matches_df['comp'] == 'Bundesliga']

## Reexplore data

See the shape of the data - rows and columns.

In [110]:
matches_df.shape

(3682, 157)

The dataset is still big enough to work with. ~800 matches got dropped.

In [112]:
matches_df.info(verbose=True, show_counts=True)
# TODO: play with date and time, since it can be a factor

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3682 entries, 1 to 4424
Data columns (total 157 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    date                            3682 non-null   object 
 1    time                            3682 non-null   object 
 2    comp                            3682 non-null   object 
 3    round                           3682 non-null   object 
 4    day                             3682 non-null   object 
 5    venue                           3682 non-null   object 
 6    result                          3682 non-null   object 
 7    gf                              3682 non-null   int64  
 8    ga                              3682 non-null   int64  
 9    opponent                        3682 non-null   object 
 10   xg                              3680 non-null   float64
 11   xga                             3680 non-null   float64
 12   poss              

`opponent` and `team` are columns that are supposed to have the same values. Check if that is the case.

In [129]:
teams = matches_df['team'].unique().tolist()
opponents = matches_df['opponent'].unique().tolist()
teams.sort()
opponents.sort()

print(len(teams))
print(teams)

print(len(opponents))
print(opponents)

27
['Arminia', 'Augsburg', 'Bayer Leverkusen', 'Bayern Munich', 'Bochum', 'Darmstadt 98', 'Dortmund', 'Dusseldorf', 'Eintracht Frankfurt', 'Freiburg', 'Greuther Furth', 'Hamburger SV', 'Hannover 96', 'Heidenheim', 'Hertha BSC', 'Hoffenheim', 'Koln', 'Mainz 05', 'Monchengladbach', 'Nurnberg', 'Paderborn 07', 'RB Leipzig', 'Schalke 04', 'Stuttgart', 'Union Berlin', 'Werder Bremen', 'Wolfsburg']
25
['Arminia', 'Augsburg', 'Bayern Munich', 'Bochum', 'Dortmund', 'Düsseldorf', 'Eint Frankfurt', 'Freiburg', 'Greuther Fürth', 'Hamburger SV', 'Hannover 96', 'Hertha BSC', 'Hoffenheim', 'Köln', 'Leverkusen', "M'Gladbach", 'Mainz 05', 'Nürnberg', 'Paderborn 07', 'RB Leipzig', 'Schalke 04', 'Stuttgart', 'Union Berlin', 'Werder Bremen', 'Wolfsburg']


Normalize values.

In [132]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    'Düsseldorf': 'Dusseldorf',
    'Eint Frankfurt': 'Eintracht Frankfurt',
    'Greuther Fürth': 'Greuther Furth',
    'Köln': 'Koln',
    'M\'Gladbach': 'Monchengladbach',
    'Nürnberg': 'Nurnberg',
    'Leverkusen': 'Bayer Leverkusen',
}

mapping = MissingDict(**map_values)
matches_df['opponent'] = matches_df['opponent'].map(mapping)

Show differences between `team` and `opponent` unique values.

In [133]:
teams = matches_df['team'].unique().tolist()
opponents = matches_df['opponent'].unique().tolist()
teams.sort()
opponents.sort()

print(set(teams) - set(opponents))

{'Darmstadt 98', 'Heidenheim'}


Both `Darmstadt 98` and `Heidenheim` are making a return in the Bundesliga so this difference is not a mistake.